<a href="https://colab.research.google.com/github/techjyo/hello-world/blob/main/Copy_of_Backlog_Demand_08062021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read Data

In [ ]:
#install these libraries
!pip install numpy
!pip install pandas
!pip install xlsxwriter
!pip install openpyxl

# Import these libraries
import numpy as np
import pandas as pd
import math
import xlsxwriter

In [ ]:
#read code- you will read in the files need to be merged together, in this example i already use the merged file- *automate
xls = pd.read_excel('/content/MOS_Report.xlsx')
US = xls[xls['WAREHOUSE_ID'] == 'US']
NL = xls[xls['WAREHOUSE_ID'] == 'NL']
JP = xls[xls['WAREHOUSE_ID'] == 'JP']
CN = xls[xls['WAREHOUSE_ID'] == 'CN']


# Clean Data

In [ ]:
#clean code-  convert columns function, change based on what columns you want to keep, drop nas
def convert(df):
  numeric_features = df.select_dtypes(include=['int64', 'float64']).columns
  categorical_features = df.select_dtypes(include=['object']).columns

# Typecast all variables that are categorical as type "category"- basically this can be used if you want to run a regression or any model on it except ignore text data for now
  for col in categorical_features.tolist():
    df[col] = df[col].astype('category')

  df['Backlog_QTY'].fillna(0, inplace=True)
  df['90d'].fillna(0, inplace=True)
  df['COGS'].fillna(0, inplace=True)
  df['AVAILABLE_QUANTITY'].fillna(0, inplace=True)
  df['Backlog_QTY'] = df['Backlog_QTY'].astype('int64')
  df['AVAILABLE_QUANTITY'] = df['AVAILABLE_QUANTITY'].astype('int64')
  df['90d'] = df['90d'].astype('int64')

In [ ]:
convert(US)
convert(NL)
convert(JP)
convert(CN)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

In [ ]:
US

,ITEM_NAME,WAREHOUSE_ID,LIST_PRICE,ROUTING_CLASS,ROUTING_SUBCLASS,COGS,ON_DEMO_QTY,NO_OF_SHIPMENTS( in 455 days),INCOMING_FROM_MANUFACTURE,Backlog_QTY,Inventory_Item_Status_Code,Migration_Product ID,End_of_Sale_Date,90d,AVAILABLE_QUANTITY,Part_Class,MONTHLY_DEMAND,MONTHLY_SUPPLY
4,A9K-SIP-700,US,82830.0,TANGIBLE,BOARDS,4901.0,0,0,0,0,NaN,NaN,NaT,0,0,A,0.0,0.0
6,ASR-920-24TZ-M,US,5025.0,TANGIBLE,SMALL CHASSIS,693.0,0,0,0,0,NaN,NaN,NaT,0,0,A,0.0,0.0
15,NCS-5502-SE,US,903600.0,TANGIBLE,LARGE CHASSIS,14198.0,0,0,0,0,NaN,NaN,NaT,0,0,A,0.0,0.0
16,ONS-XC-10G-C,US,23678.0,TANGIBLE,MEMORY,631.0,0,0,0,0,NaN,NaN,NaT,0,0,A,0.0,0.0
20,15454-SMR2-LIC,US,47063.0,TANGIBLE,BOARDS,7121.0,0,0,0,0,ENABLE-OPT,NaN,NaT,0,2,A,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15768,UCS-SD480GSAS-EV,US,2513.0,TANGIBLE,HARD DRIVES,407.0,0,0,0,0,NaN,NaN,2021-08-06,0,0,C,0.0,0.0
15769,UCS-VIC-M82-8P,US,1783.0,TANGIBLE,BOARDS,13.0,8,0,0,0,NaN,NaN,2019-02-14,0,0,C,0.0,0.0
15770,UCS-VIC-M82-8P,US,1783.0,TANGIBLE,BOARDS,13.0,8,0,0,0,NaN,NaN,2019-02-14,0,0,C,0.0,0.0
15771,VEDGE-1000-BRACK,US,111.0,NTT,BOARDS,19.0,13,0,0,0,NaN,NaN,2021-10-22,0,0,C,0.0,0.0


# Automate planning tool

In [ ]:
def createVar(df):
#creates new variables for excel to help with narrowing down choices for products to buy
  df['MOS_Tier'] = 0
  df['Backlog_Tier'] = 0
    #creating something that shows months of supply as 3 category values- 1: dire need, 2: ok, 3: not needed
  for index, row in df.iterrows():
    if (row['MONTHLY_SUPPLY'] < 4):
      row['MOS_Tier'] = 1
    elif (row['MONTHLY_SUPPLY'] < 7):
      row['MOS_Tier'] = 2
    else:
      row['MOS_Tier'] = 3
    df['MOS_Tier'][index] = row['MOS_Tier']
    if (row['MONTHLY_DEMAND'] != 0.0):
      if (row['Backlog_QTY'] / row['MONTHLY_DEMAND'] < 4):
        row['Backlog_Tier'] = 3
      elif (row['Backlog_QTY'] / row['MONTHLY_DEMAND'] < 7):
        row['Backlog_Tier'] = 2
      else:
        row['Backlog_Tier'] = 1
    df['Backlog_Tier'][index] = row['Backlog_Tier']
    df['lead_time'] = 60
    df['items_priority'] = ""
    df['replenishment_qty'] = 0
    df['replenishment_qty_demand_during_LT'] = 0
    df['cost_of_purchase']=0
    df['purchase_cost_with_demand_during_LT']=0
    df['replenishment_qty_demand_during_LT_2'] = 0
    df['purchase_cost_with_demand_during_LT_2']=0

    if (row['90d'] != 0):
      df['ratio_inc_90d'] = df['INCOMING_FROM_MANUFACTURE']/df['90d']
    else:
      df['ratio_inc_90d'] = 0
    df['ratio_inc_90d'].fillna(0, inplace=True)

In [ ]:
createVar(US)
createVar(JP)
createVar(NL)
createVar(CN)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats i

In [ ]:
def choose(df):
  #categorizes the products based on specific variables such as backlog, tier/mos
  for index, row in df.iterrows():
    if (row['Backlog_Tier'] == 1):
      row['items_priority'] = "A1"
      df['items_priority'][index] = row['items_priority']
    elif (row['Backlog_Tier'] == 2):
      row['items_priority'] = "A2"
      df['items_priority'][index] = row['items_priority']
    elif (row['Backlog_Tier'] == 3 ):
      row['items_priority'] = "A3"
      df['items_priority'][index] = row['items_priority']
    elif (row['MOS_Tier'] == 1 ):
      row['items_priority'] = "B1"
      df['items_priority'][index] = row['items_priority']
    elif (row['MOS_Tier'] == 2):
      row['items_priority'] = "B2"
      df['items_priority'][index] = row['items_priority']
    else:
      row['items_priority'] = "C"
      df['items_priority'][index] = row['items_priority']

In [ ]:
choose(US)
choose(JP)
choose(NL)
choose(CN)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == ''

In [ ]:
#sorts by priority classification, ratio- lower means more likely to choose, specific months of supply
US.sort_values(['items_priority', 'MONTHLY_SUPPLY'], ascending = [True, True], inplace = True)
NL.sort_values(['items_priority', 'MONTHLY_SUPPLY'], ascending = [True, True], inplace = True)
JP.sort_values(['items_priority', 'MONTHLY_SUPPLY'], ascending = [True, True], inplace = True)
CN.sort_values(['items_priority', 'MONTHLY_SUPPLY'], ascending = [True, True], inplace = True)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [ ]:
#check if all categories are there
US

,ITEM_NAME,WAREHOUSE_ID,LIST_PRICE,ROUTING_CLASS,ROUTING_SUBCLASS,COGS,ON_DEMO_QTY,NO_OF_SHIPMENTS( in 455 days),INCOMING_FROM_MANUFACTURE,Backlog_QTY,Inventory_Item_Status_Code,Migration_Product ID,End_of_Sale_Date,90d,AVAILABLE_QUANTITY,Part_Class,MONTHLY_DEMAND,MONTHLY_SUPPLY,MOS_Tier,Backlog_Tier,lead_time,items_priority,replenishment_qty,replenishment_qty_demand_during_LT,cost_of_purchase,purchase_cost_with_demand_during_LT,replenishment_qty_demand_during_LT_2,purchase_cost_with_demand_during_LT_2,ratio_inc_90d
4772,PWR-C1-1100WAC-P,US,1910.0,TANGIBLE,BOARDS,73.0,287,413,30,352,NaN,NaN,NaT,176,0,B,27.321538,0.0,1,1,60,A1,0,0,0,0,0,0,0
4773,PWR-C1-1100WAC-P,US,1910.0,TANGIBLE,BOARDS,73.0,287,413,30,352,NaN,NaN,NaT,176,0,B,27.321538,0.0,1,1,60,A1,0,0,0,0,0,0,0
4777,C8500-12X,US,NaN,TANGIBLE,SMALL CHASSIS,2678.0,2,6,4,12,NaN,NaN,NaT,5,0,B,0.396923,0.0,1,1,60,A1,0,0,0,0,0,0,0
4778,C8500-12X,US,NaN,TANGIBLE,SMALL CHASSIS,2678.0,2,6,4,12,NaN,NaN,NaT,5,0,B,0.396923,0.0,1,1,60,A1,0,0,0,0,0,0,0
11427,UCS-MR-X64G2RW,US,5382.0,TANGIBLE,MEMORY,322.0,24,24,0,16,NaN,NaN,NaT,0,0,C,1.587692,0.0,1,1,60,A1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15768,UCS-SD480GSAS-EV,US,2513.0,TANGIBLE,HARD DRIVES,407.0,0,0,0,0,NaN,NaN,2021-08-06,0,0,C,0.000000,0.0,1,0,60,B1,0,0,0,0,0,0,0
15769,UCS-VIC-M82-8P,US,1783.0,TANGIBLE,BOARDS,13.0,8,0,0,0,NaN,NaN,2019-02-14,0,0,C,0.000000,0.0,1,0,60,B1,0,0,0,0,0,0,0
15770,UCS-VIC-M82-8P,US,1783.0,TANGIBLE,BOARDS,13.0,8,0,0,0,NaN,NaN,2019-02-14,0,0,C,0.000000,0.0,1,0,60,B1,0,0,0,0,0,0,0
15771,VEDGE-1000-BRACK,US,111.0,NTT,BOARDS,19.0,13,0,0,0,NaN,NaN,2021-10-22,0,0,C,0.000000,0.0,1,0,60,B1,0,0,0,0,0,0,0


In [ ]:
#check amount of classified items
US['items_priority'].value_counts()

A3    4532
B1    1725
A1      48
A2      26
Name: items_priority, dtype: int64

In [ ]:
#checks budget vs cost of items to buy
#if good, the program outputs the item
#otherwise, nothing pops out
def budget(df):
  for index, row in df.iterrows():
    amount1 = math.ceil(max(row['AVAILABLE_QUANTITY'] - row['Backlog_QTY'] + row['INCOMING_FROM_MANUFACTURE'], 0))
    cost1 = row['COGS'] * amount1
    row['replenishment_qty'] = amount1
    df['replenishment_qty'][index] = row['replenishment_qty']      
    row['cost_of_purchase'] = cost1
    df['cost_of_purchase'][index] = row['cost_of_purchase'] 

    #amount2 = math.ceil(max(row['MONTHLY_DEMAND'] * 3 + row['Backlog_QTY'] + row['lead_time']/30 * row['MONTHLY_DEMAND'] - row['INCOMING_FROM_MANUFACTURE'] - row['AVAILABLE_QUANTITY'], 0))
    amount2 = math.ceil(max(row['AVAILABLE_QUANTITY'] - row['Backlog_QTY'] + row['INCOMING_FROM_MANUFACTURE'] + row['MONTHLY_DEMAND'] * 3 , 0))
    cost2 = row['COGS'] * amount2    
    row['replenishment_qty_demand_during_LT'] = amount2
    df['replenishment_qty_demand_during_LT'][index] = row['replenishment_qty_demand_during_LT']
    row['purchase_cost_with_demand_during_LT'] = cost2
    df['purchase_cost_with_demand_during_LT'][index] = row['purchase_cost_with_demand_during_LT']

    #amount3 = math.ceil(max(row['MONTHLY_DEMAND'] * 3 + row['Backlog_QTY'] + row['lead_time']/30 * row['MONTHLY_DEMAND'] - row['INCOMING_FROM_MANUFACTURE'] - row['AVAILABLE_QUANTITY'], 0))
    amount3 = math.ceil(max(row['AVAILABLE_QUANTITY'] - row['Backlog_QTY'] + row['INCOMING_FROM_MANUFACTURE'] + row['MONTHLY_DEMAND'] * 3 + (row['MONTHLY_DEMAND'] * row['lead_time'] / 60) , 0))
    cost3 = row['COGS'] * amount3    
    row['replenishment_qty_demand_during_LT_2'] = amount3
    df['replenishment_qty_demand_during_LT_2'][index] = row['replenishment_qty_demand_during_LT_2']
    row['purchase_cost_with_demand_during_LT_2'] = cost3
    df['purchase_cost_with_demand_during_LT_2'][index] = row['purchase_cost_with_demand_during_LT_2']

In [ ]:
budget(US)
budget(JP)
budget(NL)
budget(CN)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by Interacti

In [ ]:
writer = pd.ExcelWriter('newbuy_items_priority.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
US.to_excel(writer, sheet_name='US', columns=['ITEM_NAME', 'WAREHOUSE_ID', 'LIST_PRICE', 'ROUTING_CLASS',
       'ROUTING_SUBCLASS', 'Migration_Product ID',
       'End_of_Sale_Date', 'Inventory_Item_Status_Code', 'lead_time', 'COGS', 'ON_DEMO_QTY', 'AVAILABLE_QUANTITY', 'Backlog_QTY', 
       'NO_OF_SHIPMENTS( in 455 days)', 'MONTHLY_DEMAND', 'MONTHLY_SUPPLY',
       'INCOMING_FROM_MANUFACTURE', 'Part_Class', '90d', 'items_priority', 'replenishment_qty', 'cost_of_purchase',
       'replenishment_qty_demand_during_LT', 'purchase_cost_with_demand_during_LT','replenishment_qty_demand_during_LT_2','purchase_cost_with_demand_during_LT_2'], index=False)
NL.to_excel(writer, sheet_name='NL', columns=['ITEM_NAME', 'WAREHOUSE_ID', 'LIST_PRICE', 'ROUTING_CLASS',
       'ROUTING_SUBCLASS', 'Migration_Product ID',
       'End_of_Sale_Date', 'Inventory_Item_Status_Code', 'lead_time', 'COGS', 'ON_DEMO_QTY', 'AVAILABLE_QUANTITY', 'Backlog_QTY', 
       'NO_OF_SHIPMENTS( in 455 days)', 'MONTHLY_DEMAND', 'MONTHLY_SUPPLY',
       'INCOMING_FROM_MANUFACTURE', 'Part_Class', '90d', 'items_priority', 'replenishment_qty', 'cost_of_purchase',
       'replenishment_qty_demand_during_LT', 'purchase_cost_with_demand_during_LT','replenishment_qty_demand_during_LT_2','purchase_cost_with_demand_during_LT_2'], index=False)
JP.to_excel(writer, sheet_name='JP', columns=['ITEM_NAME', 'WAREHOUSE_ID', 'LIST_PRICE', 'ROUTING_CLASS',
       'ROUTING_SUBCLASS', 'Migration_Product ID',
       'End_of_Sale_Date', 'Inventory_Item_Status_Code', 'lead_time', 'COGS', 'ON_DEMO_QTY', 'AVAILABLE_QUANTITY', 'Backlog_QTY', 
       'NO_OF_SHIPMENTS( in 455 days)', 'MONTHLY_DEMAND', 'MONTHLY_SUPPLY',
       'INCOMING_FROM_MANUFACTURE', 'Part_Class', '90d', 'items_priority', 'replenishment_qty', 'cost_of_purchase',
       'replenishment_qty_demand_during_LT', 'purchase_cost_with_demand_during_LT','replenishment_qty_demand_during_LT_2','purchase_cost_with_demand_during_LT_2'], index=False)
CN.to_excel(writer, sheet_name='CN', columns=['ITEM_NAME', 'WAREHOUSE_ID', 'LIST_PRICE', 'ROUTING_CLASS',
       'ROUTING_SUBCLASS', 'Migration_Product ID',
       'End_of_Sale_Date', 'Inventory_Item_Status_Code', 'lead_time', 'COGS', 'ON_DEMO_QTY', 'AVAILABLE_QUANTITY', 'Backlog_QTY', 
       'NO_OF_SHIPMENTS( in 455 days)', 'MONTHLY_DEMAND', 'MONTHLY_SUPPLY',
       'INCOMING_FROM_MANUFACTURE', 'Part_Class', '90d', 'items_priority', 'replenishment_qty', 'cost_of_purchase',
       'replenishment_qty_demand_during_LT', 'purchase_cost_with_demand_during_LT','replenishment_qty_demand_during_LT_2','purchase_cost_with_demand_during_LT_2'], index=False)

# Close the Pandas Excel writer and output the Excel file so that you can have a final merged file
writer.save()